</div>
<div align="left">
  <img src="img/abstract.png" width="400" alt="Funny little diagram">
  <p><em> Evolve nodes, evolve plans, and learn from the best performing ones.</em></p>
</div>
<div align="center">
</em></p>
</div>

#### Node Initialization (Refactoring ...)

In [1]:
from methods.llm import get_async_vllm_endpoint
import os 

endpoint_id = "vllm-vbzs7iyoydoidq"
api_key = os.environ["RUNPOD_API_KEY"]

get_endpoint_response = get_async_vllm_endpoint(endpoint_id, api_key) # Unlimited LLM endpoints
get_endpoint_response(["Hello, world!"] * 12)

Could not load vllm class, check CUDA support and GPU RAM size


["Hello, world!  It's great to be talking to you.\n\nAs a Turing Award winner, I have had the privilege of working on some of the most groundbreaking projects in computer science. From developing new algorithms to inventing novel data structures, my work has been focused on pushing the boundaries of what is possible with computers.\n\nI must say, it's a thrill to be talking to you today. I'm excited to share my knowledge and experience with you, and to hear about your interests and passions.\n\nSo, what would you like to talk about? Are you interested in learning about a specific area of computer science, or perhaps you have a particular problem you'd like to solve? I'm all ears!",
 "Nice to meet you.  I'm happy to chat with you today.  I received the Turing Award in 2002 for my work on the development of the Linux operating system and my contributions to the open-source movement.  It was a truly incredible honor, and I'm grateful to have had the opportunity to work on something that h

In [2]:
from methods.meta_prompt import MetaPrompt, PromptMode
from methods.evolnode import EvolNode
from methods.llm import get_groq_response, get_claude_response

# Code + Compilor Task
# mp = MetaPrompt("Search for age of a celebrity.", "get_celeb_age", ["name"], ["age"], ["str"], ["int"], PromptMode.CODE)
# Prompt + LLM Task
mp = MetaPrompt("Get the age of a celebrity.", "get_celeb_age", ["name"], ["age"], ["str"], ["int"], PromptMode.PROMPT) # 

test_cases = [
    ({"name": "Dilireba"}, {"age": 32}),
    ({"name": "ChengXiao"}, {"age": 26})
]

test_inputs = [c[0] for c in test_cases]

node = EvolNode(mp, None, None, get_response=get_endpoint_response, test_cases=test_cases) # setting manual test cases
# node = EvolNode(mp, None, None, get_response=get_groq_response) # automatic generation of test cases 


# node.evolve("i1", replace=True, batch_size=3, num_runs=2)

# print("Inspection on the generated code: \n", node.code)

# import time 
# time.sleep(3)

# input_dict = test_inputs[0]
# output_dict = node(input_dict) # Could still have error due to unsuccessful parsing of output
# print("Output from the code: \n", output_dict)

In [3]:
self = node 
replace = True 
method = "i1"
parents = None 
batch_size = 3

self.query_nodes(ignore_self=replace, self_func_name=self.meta_prompt.func_name)
        
# Evolve many times
reasonings, codes = self._evolve(method, parents, batch_size=batch_size) # This is fine (nice!)

In [17]:
max_tries = 3 
num_runs = 2 

# fitnesses, err_msgs  = self._evaluate_fitness(codes=codes, max_tries=max_tries, num_runs=num_runs)


if len(codes) == 0: 
    codes = [self.code] 

if test_cases is None:
    test_cases = self.test_cases

if self.meta_prompt.mode == PromptMode.PROMPT:
    num_runs = min(2, num_runs) # sanity check against stochastic nature of prompt-based node

test_inputs = [t[0] for t in test_cases]

# if self.meta_prompt.mode == PromptMode.CODE: 
output_per_code_per_test, error_per_code_per_test = self.call_code_function_parallel(test_cases, codes)
# elif self.meta_prompt.mode == PromptMode.PROMPT:
output_per_code_per_test, errors_per_code_per_test = self.call_prompt_function_parallel(test_inputs, codes, max_tries)

In [23]:
from typing import List, Dict, Any
from collections import defaultdict
import types
from methods.meta_execute import extract_json_from_text

def call_func_prompt_parallel(input_dicts: List[Dict[str, Any]], codes: List[str], max_tries: int, get_response: callable):
    """ 
    Parallel calling of prompt function
    """
    outputs_per_code_per_test = defaultdict(lambda: defaultdict(list))
    errors_per_code_per_test = defaultdict(lambda: defaultdict(list))
    
    prompts = []
    input_indices = []
    for (input_index, input_dict) in enumerate(input_dicts):
        for (code_index, code) in enumerate(codes):
            mod = types.ModuleType('dynamic_module')
            exec(code, mod.__dict__)
            func_name = "generate_prompt"
            prompt_func = mod.__dict__[func_name]
            try:
                prompt = prompt_func(**input_dict)
                prompts.append(prompt)
                input_indices.append((input_index, code_index))
            except Exception as e:
                errors_per_code_per_test[code_index][input_index].append(str(e))
            
    responses = get_response(prompts)
    
    for (response, (input_index, code_index)) in zip(responses, input_indices):
        try:
            output_dict = extract_json_from_text(response)
            outputs_per_code_per_test[code_index][input_index].append(output_dict)
        
        except Exception as e:
            errors_per_code_per_test[code_index][input_index].append(f"Failed to parse LLM response: {response}\nError: " + str(e))
    
    output_per_code_per_test = defaultdict(lambda: defaultdict(dict))
    for code_index in outputs_per_code_per_test:
        for input_index in outputs_per_code_per_test[code_index]:
            for output_dict in outputs_per_code_per_test[code_index][input_index]:
                if output_dict != {}:   # Keep non-empty output dict
                    output_per_code_per_test[code_index][input_index] = output_dict
                
    return output_per_code_per_test, errors_per_code_per_test

</div>
<div align="center">
  <img src="img/Project-Nirvana-evolve.gif" width="500" alt="Fourier reconstruction convergence">
  <p><em> Evolve a population of nodes. </em></p>
</div>

In [4]:
# Population building phase ... 
from methods.llm import get_groq_response
from methods.meta_prompt import MetaPrompt, PromptMode
from methods.population import Evolution

mp = MetaPrompt("Get the age of a celebrity.", "get_celeb_age", ["name"], ["age"], ["str"], ["int"], PromptMode.PROMPT) # 

test_cases = [
    ({"name": "Dilireba"}, {"age": 32}),
    ({"name": "ChengXiao"}, {"age": 26})
]

evo = Evolution(pop_size=3, meta_prompt=mp, get_response=get_groq_response, 
                test_cases=test_cases, max_attempts=3, num_eval_runs=2,
                load=True)

strategies = ["m2"] # ["i1", "i1", "m2", "e2"]
evo.get_offspring(strategies)

evo.chat("How effective is the current evolution strategy? What improvement has it made in terms of fitness, and in terms of the implementation?",
         get_claude_response) 

# code-based check 
print(evo.population_info)

Evaluating fitness: 100%|██████████| 4/4 [00:00<00:00, 6041.49it/s]


--- Replacing with new node
 - Attempt 1 failed. Fitness: 0.00. Error:  This node has no idea of what's going on
--- Compiled 0 out of 4 test cases
--- Passed 0 out of 4 test cases
Input: {'name': 'Dilireba'}, Output is missing or of wrong type, Expected: {'age': 32}
Input: {'name': 'ChengXiao'}, Output is missing or of wrong type, Expected: {'age': 26}
Input: {'name': 'Dilireba'}, Output is missing or of wrong type, Expected: {'age': 32}
Input: {'name': 'ChengXiao'}, Output is missing or of wrong type, Expected: {'age': 26}

Error Message:
--- Calling Prompt Function Error:
unexpected '{' in field name--- Calling Prompt Function Error:
unexpected '{' in field name--- Calling Prompt Function Error:
unexpected '{' in field name--- Calling Prompt Function Error:
unexpected '{' in field name


Evaluating fitness: 100%|██████████| 4/4 [00:02<00:00,  1.77it/s]


--- Replacing with new node
 - Attempt 2 failed. Fitness: 0.75. Error:  Function runs with success rate: 75.0%, runs correctly with rate: 75.0%
--- Compiled 3 out of 4 test cases
--- Passed 3 out of 4 test cases
Input: {'name': 'ChengXiao'}, Output is missing or of wrong type, Expected: {'age': 26}

Error Message:
--- Calling Prompt Function Error:
Failed to parse LLM response: No JSON structure found in the provided text.


Evaluating fitness: 100%|██████████| 4/4 [00:07<00:00,  1.98s/it]


--- Replacing with new node
 - Attempt 3 failed. Fitness: 0.75. Error:  Function runs with success rate: 75.0%, runs correctly with rate: 75.0%
--- Compiled 3 out of 4 test cases
--- Passed 3 out of 4 test cases
Input: {'name': 'ChengXiao'}, Output is missing or of wrong type, Expected: {'age': 26}

Error Message:
--- Calling Prompt Function Error:
Failed to parse LLM response: No JSON structure found in the provided text.
Evolution failed after 3 attempts.
Based on the provided information, let me analyze the effectiveness of the current evolution strategy:

1. Fitness Improvement:
- Initial best fitness: 1.0
- Current best fitness after evolution: 0.75
- This actually shows a decrease in fitness, which suggests that the evolution strategy might not be improving the solution in terms of fitness metrics.

2. Implementation Improvements:
Despite the lower fitness score, there are several notable improvements in the implementation:

a) Added Functionality:
- Introduced the `search_google


</div>
<div align="center">
  <img src="https://github.com/user-attachments/assets/af98faeb-66d6-4278-af86-67d668d1954e" width="900" alt="Fourier reconstruction convergence">
  <p><em> Plan, and evolve the plans. </em></p>
</div>


In [1]:
from methods.llm import get_async_vllm_endpoint
import os 

endpoint_id = "vllm-vbzs7iyoydoidq"
api_key = os.environ["RUNPOD_API_KEY"]

get_endpoint_response = get_async_vllm_endpoint(endpoint_id, api_key) # Unlimited LLM endpoints
get_endpoint_response(["Hello, world!"] * 12)

Could not load vllm class, check CUDA support and GPU RAM size


["Nice to meet you. As a Turing Award winner, I've had the privilege of working on some groundbreaking projects in the field of computer science. I've contributed to the development of algorithms, data structures, and programming languages that have had a significant impact on the way we design and build software systems.\n\nMy research has focused on the intersection of computer science and artificial intelligence, and I've been fortunate enough to have made some notable contributions in this area. From the development of new machine learning algorithms to the creation of more efficient and scalable data structures, my work has aimed to push the boundaries of what is possible in the field of computer science.\n\nBut I'm not just about the tech – I'm also passionate about inspiring the next generation of computer scientists and engineers. I believe that computer science has the power to transform the world, and I'm dedicated to helping others learn and grow in this field.\n\nSo, what b

In [2]:
from methods.llm import get_claude_response, get_groq_response
from methods.diagram import visualize_plan_dict
from methods.meta_prompt import MetaPlan
from methods.evolnode import PlanNode


# Initialize PlanNode 
mp = MetaPlan("Get the age of celebrity.", "get_celeb_age", ["name"], ["age"], ["str"], ["int"])
plan = PlanNode(mp, get_endpoint_response)

# i1 evolution of plan
plan_dicts, err_msg = plan.evolve_plan_dict_parallel(method="i1", batch_size=20) # Batch_size of 100 gives no slow-down
visualize_plan_dict(plan_dicts[0], plan.meta_prompt.task)

 :: Evolving 1 plans in parallel...
 :: Pseudo-code generated for each plan
 :: Plan_dict generated for each plan


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
success: successfully compiled d2_output/plan_graph.d2 to d2_output/plan_graph.png in 185.365417ms
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explici

In [3]:
# Manual input on main-node test cases 
main_test_cases = [
    ({"name": "Dilireba"}, {"age": 32}),
    ({"name": "ChengXiao"}, {"age": 26})
]

plan.spawn_test_cases_parallel(main_test_cases) #  pinned test cases generation

Spawned 2 test cases for all sub-nodes


(True,
 "\nError in spawning test cases: JsonDecodeError : \nExpecting ',' delimiter: line 66 column 30 (char 1829)AstLiteralError : \ninvalid syntax (<unknown>, line 66)\nOutput mismatch for input {'name': 'ChengXiao'}, expected {'age': 26}, got {'age': 30}\nError in spawning test cases: JsonDecodeError : \nExpecting value: line 1 column 1 (char 0)AstLiteralError : \ninvalid syntax (<unknown>, line 1)\nError in spawning test cases: JsonDecodeError : \nExpecting value: line 1 column 1 (char 0)AstLiteralError : \ninvalid syntax (<unknown>, line 1)\nOutput mismatch for input {'name': 'Dilireba'}, expected {'age': 32}, got {'age': 31}\nOutput mismatch for input {'name': 'Dilireba'}, expected {'age': 32}, got {'age': 31}\nOutput mismatch for input {'name': 'ChengXiao'}, expected {'age': 26}, got {'age': 25}\nOutput mismatch for input {'name': 'Dilireba'}, expected {'age': 32}, got {'age': 49}\nOutput mismatch for input {'name': 'ChengXiao'}, expected {'age': 26}, got {'age': 34}\nError in 

In [4]:
plan.test_cases_dict

{'search_google': [({'name': 'Dilireba'},
   {'result': 'Dilireba is a Chinese actress born in 1992.'}),
  ({'name': 'ChengXiao'}, {'result': "ChengXiao's Wikipedia page"})],
 'get_birth_year': [({'google_result': 'Dilireba is a Chinese actress born in 1992.'},
   {'age': 32}),
  ({'google_result': 'ChengXiao was born on January 5, 1997.'}, {'age': 26})]}

In [ ]:
plan.evolve_sub_nodes() # sub-node evolution

# Bug above
# ---> 86 is_aligned, error_msg_delta = metric(pred_value, target_value)
#      87 if not is_aligned:
#      88     error_msg += error_msg_delta + "\n"

# TypeError: 'tuple' object is not callable